In [198]:
import pandas as pd
import re
from __future__ import division

In [21]:
im= pd.read_csv("./imdb.csv");
rt= pd.read_csv("./rotten_tomatoes.csv");
print('The imdb shape: %d x %d' % im.shape)
print('The rt shape: %d x %d' % im.shape)

The imdb shape: 6407 x 13
The rt shape: 6407 x 13


In [211]:

def ngrams(string, n=2):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


def split_ngram(string,n=2):
        """Normalize string and split in n-grams constaing n characters."""
        string_ = string.lower()
        string = [string_[i:i+n] for i in range(0, len(string_), n)]
        return string
    

def dice_coefficient(a, b,n=2):
    if isinstance(a,str) & isinstance(b,str):
        a = split_ngram(a,n)
        b = split_ngram(b,n)
        
    """dice coefficient 2nt/na + nb."""
    a_bigrams = set(a)
    b_bigrams = set(b)
    overlap = len(a_bigrams & b_bigrams)
    return overlap * 2.0/(len(a_bigrams) + len(b_bigrams))


In [52]:

rt.head(0)


,Id,Name,Year,Release Date,Director,Creator,Actors,Cast,Language,Country,Duration,RatingValue,RatingCount,ReviewCount,Genre,Filming Locations,Description


In [53]:
im.head(0)


,Id,Name,YearRange,ReleaseDate,Director,Creator,Cast,Duration,RatingValue,ContentRating,Genre,Url,Description


In [253]:
#Im : RT
GT = {
      "Name":"Name",
      "ReleaseDate":"Release Date",
      "RatingValue":"RatingValue",
      "Director":"Director",
      "Creator":"Creator",
      "YearRange":"Year",
      "Genre":"Genre",
      "Duration":"Duration",
      "Cast":"Cast",
    "Description":"Description"
     }

In [254]:
dice_coefficient("Name","Nameee")

0.8

In [328]:
def get_mapping(table_a,table_b,treshold=0.5):
    A =[ str(a) for a in table_a.columns]
    B =[ str(b) for b in table_a.columns]
    print (A.remove('Id'),B.remove('Id'))
    rt_cl = [[ngrams(cl),cl]for cl in A]
    im_cl = [[ngrams(cl),cl] for cl in B]
    df = pd.DataFrame(columns=["rt","im","coeff"])
    for r,r_ in rt_cl:
        for i,i_ in im_cl:
            dc = dice_coefficient(r,i)
            if dc > 0:
                df = df.append(pd.Series([r_,i_,dc],index=["rt","im","coeff"]),ignore_index=True)
    tresh = df.loc[df['coeff'] >= treshold];
    #tresh.groupby(['im','rt'], sort=False)['rt','coeff'].max()
    return dict(zip(tresh["im"],tresh['rt'])) 


    

In [329]:
df

,rt,im,coeff
0,Id,Id,1.000000
1,Name,Name,1.000000
2,Year,YearRange,0.545455
3,Year,ReleaseDate,0.153846
4,Year,Creator,0.222222
5,Release Date,YearRange,0.105263
6,Release Date,ReleaseDate,0.857143
7,Release Date,Creator,0.235294
8,Release Date,Cast,0.142857
9,Release Date,Duration,0.111111


In [330]:
tresh = df.loc[df['coeff'] > 0.7]
tresh.groupby(['rt',"im"], sort=True)['coeff'].max()


rt            im         
Cast          Cast           1.000000
Creator       Creator        1.000000
Description   Description    1.000000
Director      Director       1.000000
Duration      Duration       1.000000
Genre         Genre          1.000000
Id            Id             1.000000
Name          Name           1.000000
RatingValue   RatingValue    1.000000
Release Date  ReleaseDate    0.857143
Name: coeff, dtype: float64

In [331]:
get_mapping(rt,im,0.8)

(None, None)


{'Actors': 'Actors',
 'Cast': 'Cast',
 'Country': 'Country',
 'Creator': 'Creator',
 'Description': 'Description',
 'Director': 'Director',
 'Duration': 'Duration',
 'Filming Locations': 'Filming Locations',
 'Genre': 'Genre',
 'Language': 'Language',
 'Name': 'Name',
 'RatingCount': 'RatingCount',
 'RatingValue': 'RatingValue',
 'Release Date': 'Release Date',
 'ReviewCount': 'ReviewCount',
 'Year': 'Year'}

In [ ]:
GT.values()
def compute_recall(mapping):
    total_found =len(mapping.items())
    return float(total_found/len(GT.items()))
                 
def compute_precision(mapping):
    precision = set(mapping.items()) & set(GT.items())
    true_matches = len(precision)
    total_found =len(mapping.items())
    return float(true_matches/total_found)

x = get_mapping(rt,im,0.8)
compute_precision(x)
compute_recall(x)

In [317]:
set(x.items())


{('Actors', 'Actors'),
 ('Cast', 'Cast'),
 ('Country', 'Country'),
 ('Creator', 'Creator'),
 ('Description', 'Description'),
 ('Director', 'Director'),
 ('Duration', 'Duration'),
 ('Filming Locations', 'Filming Locations'),
 ('Genre', 'Genre'),
 ('Language', 'Language'),
 ('Name', 'Name'),
 ('RatingCount', 'RatingCount'),
 ('RatingValue', 'RatingValue'),
 ('Release Date', 'Release Date'),
 ('ReviewCount', 'ReviewCount'),
 ('Year', 'Year')}

In [315]:
GT.items()

[('ReleaseDate', 'Release Date'),
 ('Director', 'Director'),
 ('Cast', 'Cast'),
 ('Name', 'Name'),
 ('Creator', 'Creator'),
 ('Genre', 'Genre'),
 ('Description', 'Description'),
 ('Duration', 'Duration'),
 ('YearRange', 'Year'),
 ('RatingValue', 'RatingValue')]

In [316]:
set(x.items()) & set(GT.items())

{('Cast', 'Cast'),
 ('Creator', 'Creator'),
 ('Description', 'Description'),
 ('Director', 'Director'),
 ('Duration', 'Duration'),
 ('Genre', 'Genre'),
 ('Name', 'Name'),
 ('RatingValue', 'RatingValue')}